# **Import**

In [2]:
import phoenix as px
import llama_index
from llama_index.callbacks import arize_phoenix
from sqlalchemy import create_engine
from glob import glob
import os

C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# not work with JSON Query Engine
px.launch_app()
llama_index.core.set_global_handler("arize_phoenix")

In [4]:
os.environ["OPENAI_API_KEY"] = ""
os.environ["TAVILY_API_KEY"] = ""
os.environ["COHERE_API_KEY"] = ""

In [5]:
engine = create_engine(f"sqlite:///path_to_db/my_database_no_agg.db")

In [6]:
from llama_index.core import SQLDatabase

sql_database = SQLDatabase(engine)

# **Query Engine**

In [7]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

In [8]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo")

In [9]:
sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
)

In [10]:
# sql_query_engine.query("In table base how many rows in target that have value = 1")

In [11]:
# sql_query_engine.query("what is all of the different value of person sex")

# **MAKE TABLE RETREIVER**

## **Create json Table Schema**  

In [50]:
import os
import json

# Specify the folder path where the JSON files are located
folder_path = "path_to_/TableInfo"

# Create an empty list to store the data
table_infos = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a JSON file
    if filename.endswith(".json"):
        # Construct the full file path
        file_path = os.path.join(folder_path, filename)
        
        # Open the JSON file and load its contents
        with open(file_path, "r") as file:
            data = json.load(file)
            
        # Append the data to the list
        table_infos.append(data)

# Print the resulting list
print(table_infos)

[{'table_name': 'applprev', 'table_summary': 'The table contains information about case_id, max_actualdpd_943P, max_annuity_853A, max_credacc_actualbalance_314A, max_credacc_credlmt_575A, and other related data for different cases.'}, {'table_name': 'base', 'table_summary': 'The table contains case_id, date_decision, MONTH, WEEK_NUM, and target columns with corresponding data.'}, {'table_name': 'credit_bureau_a', 'table_summary': "The table contains dept 1 information about credit bureau of each case_id from datasource 'a'. Contains columns of case IDs and various credit-related attributes such as credit limits, debt outstanding, overdue amounts, installment amounts, dates, financial institutions, and more."}, {'table_name': 'credit_bureau_b', 'table_summary': "The table contains dept 1 information about credit bureau of each case_id from datasource 'b'. Contains columns of case IDs and various credit-related attributes such as maximum amounts, credit limits, debt values, installment a

In [51]:
with open("table_desciption.json", "w") as file:
    json.dump(table_infos, file, indent=4)

## **Table Retreiver**

In [ ]:
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import SQLDatabase, VectorStoreIndex
engine = create_engine(f"sqlite:///path_to/my_database_no_agg.db")
sql_database = SQLDatabase(engine)

table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    SQLTableSchema(table_name=t["table_name"], context_str=t["table_summary"])
    for t in table_infos
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
# obj_retriever = obj_index.as_retriever(similarity_top_k=3)

In [13]:
from llama_index.core.indices.struct_store import SQLTableRetrieverQueryEngine
table_retreiver_query_engine = SQLTableRetrieverQueryEngine(
    sql_database,
    obj_index.as_retriever(similarity_top_k=3),
)

In [14]:
# response = table_retreiver_query_engine.query("descibe tax registry table?")
# response

# **Sub Query**

In [15]:
import nest_asyncio

nest_asyncio.apply()

In [16]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import Settings

In [17]:
# Using the LlamaDebugHandler to print the trace of the sub questions
# captured by the SUB_QUESTION callback event type
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

Settings.callback_manager = callback_manager

In [18]:
# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=table_retreiver_query_engine,
        metadata=ToolMetadata(
            name="credit_data",
            description="Database contain the value that related to credit prediction",
        ),
    ),
]

subquery_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

In [19]:
subquery_engine.query("what is the case_id that has the most credit limit. then what how many person has that case_id")

# **Auto Vector Query Engine**

In [39]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
)
from llama_index.core import SummaryIndex

In [40]:
documents = SimpleDirectoryReader("../../misc/data_description").load_data()

In [41]:
documents

[Document(id_='81cf698b-312b-43f8-ab4a-2eee990269dd', embedding=None, metadata={'file_path': 'D:\\Coding\\KBTG_HACK\\langchain\\data_description\\data_description.txt', 'file_name': 'data_description.txt', 'file_type': 'text/plain', 'file_size': 38933, 'creation_date': '2024-05-23', 'last_modified_date': '2024-05-23'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Table Description\r\nThis dataset contains a large number of tables as a result of utilizing diverse data sources and the varying levels of data aggregation used while preparing the dataset\r\n\r\nBase tables\r\nBase tables store the basic information about the observation and case_id. This is a unique identification of every observation and you need to use it to join the other tables to base

In [42]:
from llama_index.core import Settings

# initialize settings (set chunk size)
Settings.chunk_size = 1024
nodes = Settings.node_parser.get_nodes_from_documents(documents)

In [43]:
# initialize storage context (by default it's in-memory)
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

In [44]:
summary_index = SummaryIndex(nodes, storage_context=storage_context)
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)

In [47]:
from llama_index.llms.openai import OpenAI
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores import MetadataInfo, VectorStoreInfo
from llama_index.core.query_engine import RetrieverQueryEngine


vector_store_info = VectorStoreInfo(
    content_info="document about all of the Table information.",
    metadata_info=[
        MetadataInfo(
            name="title", type="str", description="information about the table"
        ),
    ],
)
vector_auto_retriever = VectorIndexAutoRetriever(
    vector_index, vector_store_info=vector_store_info
)

retriever_query_engine = RetrieverQueryEngine.from_args(
    vector_auto_retriever, llm=OpenAI(model="gpt-3.5-turbo")
)

In [48]:
from llama_index.core.tools import QueryEngineTool

vector_tool = QueryEngineTool.from_defaults(
    query_engine=retriever_query_engine,
    description=(
        f"Useful for answering semantic questions about the table, e.g. describtions, columns describtions"
    ),
)

In [ ]:
from llama_index.core.query_engine import SQLAutoVectorQueryEngine

auto_query_engine = SQLAutoVectorQueryEngine(
    sql_tool, vector_tool, llm=OpenAI(model="gpt-3.5-turbo")
)

In [31]:
auto_query_engine.query("what is the maximum value of maximum amounts across the tax registries table")

Querying SQL database: The question requires obtaining a value from a table using SQL commands, which is necessary to find the maximum value of maximum amounts across the tax registries table.
SQL query: SELECT MAX(amount_4917619A) AS max_amount
FROM tax_registry_b
UNION
SELECT MAX(amount_4527230A) AS max_amount
FROM tax_registry_a
UNION
SELECT MAX(pmtamount_36A) AS max_amount
FROM tax_registry_c;
SQL response: The maximum value of maximum amounts across the tax registries table is $344,250.00.
Transformed query given SQL response: None


Response(response='The maximum value of maximum amounts across the tax registries table is $344,250.00.', source_nodes=[NodeWithScore(node=TextNode(id_='ec6bd576-d0a4-4947-a87f-244f0332600b', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='[(87115.6,), (344250.0,)]', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'ec6bd576-d0a4-4947-a87f-244f0332600b': {}, 'sql_query': 'SELECT MAX(amount_4917619A) AS max_amount\nFROM tax_registry_b\nUNION\nSELECT MAX(amount_4527230A) AS max_amount\nFROM tax_registry_a\nUNION\nSELECT MAX(pmtamount_36A) AS max_amount\nFROM tax_registry_c;', 'result': [(87115.6,), (344250.0,)], 'col_keys': ['max_amount']})

In [33]:
from llama_index.core.indices.struct_store import SQLTableRetrieverQueryEngine
table_retreiver_query_engine = SQLTableRetrieverQueryEngine(
    sql_database,
    retriever_query_engine,
)

In [ ]:
table_retreiver_query_engine.query("what is the maximum value of maximum amounts across the tax registries table")

# **FLARE**

In [76]:
from llama_index.core.query_engine import FLAREInstructQueryEngine

flare_query_engine = FLAREInstructQueryEngine(
    query_engine=vector_index.as_query_engine(similarity_top_k=2),
    max_iterations=7,
    verbose=True,
)

In [79]:
response = flare_query_engine.query("If I were to create a ml which table and columns I should be include.")

Query: If I were to create a ml which table and columns I should be include.
Current response: 
Lookahead response: [Search(What are the essential components of a machine learning model?)]
Updated lookahead response: The essential components of a machine learning model typically include data preprocessing, feature selection, model selection, model training, model evaluation, and model deployment. These components are crucial for developing an effective and accurate machine learning model.
Current response:  The essential components of a machine learning model typically include data preprocessing, feature selection, model selection, model training, model evaluation, and model deployment. These components are crucial for developing an effective and accurate machine learning model.
Lookahead response: [Search(What are the essential components of a machine learning model?)]
Updated lookahead response: The essential components of a machine learning model typically include features (predicto

# **Multi-Step Query Engine**

In [13]:

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [14]:
documents = SimpleDirectoryReader("../../misc/data_description").load_data()

In [16]:

gpt35 = OpenAI(temperature=0, model="gpt-3.5-turbo")

In [17]:
index = VectorStoreIndex.from_documents(documents)

In [ ]:
# set Logging to DEBUG for more detailed outputs
from llama_index.core.query_engine import MultiStepQueryEngine

query_engine = index.as_query_engine(llm=gpt35)
query_engine = MultiStepQueryEngine(
    query_engine=query_engine,
    query_transform=step_decompose_transform,
    index_summary=index_summary,
)
response_gpt4 = query_engine.query(
    "Who was in the first batch of the accelerator program the author"
    " started?",
)

# **ROUTER**

## Setup Tools

In [45]:
from llama_index.core.tools import QueryEngineTool


table_tool = QueryEngineTool.from_defaults(
    query_engine=table_retreiver_query_engine,
    description=(
        "Useful when the query is simple or have 1 question and require to run SQL command to get information from the tables in the database"
    ),
)

subquery_tool = QueryEngineTool.from_defaults(
    query_engine=subquery_engine,
    description=(
        "Useful when the queries is complicate or have multiple questions and require to run sql command to get information from the tables in the database"
    ),
)

In [46]:
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector, LLMMultiSelector
from llama_index.core.selectors import (
    PydanticMultiSelector,
    PydanticSingleSelector,
)


query_engine = RouterQueryEngine(
    selector=PydanticSingleSelector.from_defaults(),
    query_engine_tools=[
        table_tool,
        subquery_tool,
    ],
)

In [47]:
query_engine.query("what is the case_id that has the most credit limit. then what how many person has that case_id")

Generated 2 sub questions.
[credit_data] Q: What is the case_id with the highest credit limit?
[credit_data] Q: How many persons have the case_id with the highest credit limit?
[credit_data] A: The case_id with the highest credit limit is 839373, with a credit limit of 796,800,000.0.
[credit_data] A: There is only 1 person with the case_id that has the highest credit limit.
**********
Trace: query
    |_CBEventType.QUERY -> 7.659772 seconds
      |_CBEventType.LLM -> 1.452533 seconds
      |_CBEventType.QUERY -> 6.182853 seconds
        |_CBEventType.LLM -> 1.197857 seconds
        |_CBEventType.SUB_QUESTION -> 3.96513 seconds
          |_CBEventType.TEMPLATING -> 0.0 seconds
          |_CBEventType.LLM -> 2.167574 seconds
          |_CBEventType.SYNTHESIZE -> 0.855232 seconds
            |_CBEventType.TEMPLATING -> 0.0 seconds
            |_CBEventType.LLM -> 0.846272 seconds
        |_CBEventType.SUB_QUESTION -> 3.18354 seconds
          |_CBEventType.TEMPLATING -> 0.0 seconds
      

Response(response='The case_id with the highest credit limit is 839373. There is only 1 person with this case_id.', source_nodes=[NodeWithScore(node=TextNode(id_='b0a9f008-cf2e-4560-b334-513f7e3d939e', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Sub question: What is the case_id with the highest credit limit?\nResponse: The case_id with the highest credit limit is 839373, with a credit limit of 796,800,000.0.', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None), NodeWithScore(node=TextNode(id_='eec3c80a-5850-4db8-8887-a6928907e7ca', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Sub question: How many persons have the case_id with the highest credit limit?\nResponse: There is only 1 person with the case_id that has the highest credit limit